In [36]:
# Fix the utilization on 96% and vary the number of servers
ev_arrival_rate = 40 / 60  # EV per hour
energy_req_rate = 50 / 60  # KW per hour

# Create a list with EVSE's
sim_evse = range(1, 10)
sim_time = 50000  # 3000 minutes = 50 hours
sim_reps = 30  # 20 simulations

In [31]:
# Setup file for the MMC simulation
# packages
import salabim as sim
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datetime import datetime
from scipy.special import factorial

# https://brandfetch.com/enexis.nl?view=library&library=default&collection=colors

ENEXIS_G_0 = '#B6D300'
ENEXIS_G_1 = '#BDDB00'
ENEXIS_G_2 = '#A3C100'
ENEXIS_P_0 = '#DA1369'
ENEXIS_P_1 = '#EE6DAE'
ENEXIS_P_2 = '#DF0073'
ENEXIS_B_0 = '#04296C'

# Create a string with the simulation session name
sim_session_name = (
    "df_EVSE_"
    + str(EVSES[0])
    + "-"
    + str(EVSES[-1])
    + "_time_"
    + str(sim_time)
    + "_reps_"
    + str(sim_reps)
    + ".csv"
)

print(sim_session_name)

df_EVSE_1-9_time_50000_reps_20.csv


In [32]:
# simulation code for MMC queue
# function to run simulation
def sim_facility(
    ev_arrival_rate,  # EV's per hour
    energy_req_rate,  # KW per hour
    number_of_EVSE=1,  # number of EVSE's
    sim_time=50000,
    time_unit="minutes",
    random_seed=123456,
    run=1,
):
    # Generator which creates cars
    class EV_Generator(sim.Component):
        # setup method is called when the component is created
        # and is used to initialize the component
        # switch off monitoring for mode and status
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            while True:
                EV()
                self.hold(inter_arr_time_distr.sample())

    class EV(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)

        def process(self):
            self.enter(waitingline)
            for EVSE in EVSE_facility:
                if EVSE.ispassive():
                    EVSE.activate()
                    break  # activate at most one charging station
            self.passivate()

    class EVSE(sim.Component):
        def setup(self):
            self.mode.monitor(False)
            self.status.monitor(False)
            self.length = sim.Monitor(
                name="length", monitor=True, level=True, type="int32"
            )
            self.length_of_stay = sim.Monitor(
                name="length_of_stay", monitor=True, level=False, type="float"
            )
            self.power_mon = sim.Monitor(
                name="power.", monitor=True, level=True, type="float"
            )
            self.power = 1.0

        def process(self):
            while True:
                self.length.tally(0)
                while len(waitingline) == 0:
                    self.set_mode("IDLE")
                    self.passivate(mode="IDLE")
                self.car = waitingline.pop()
                # wf = app.now()
                energy_request = energy_request_distr.sample()
                charging_time = energy_request / self.power
                # monEVSE_IDLE.tally(wf - ws)
                # monEVSE_PROC.tally(charging_time)
                self.length.tally(1)
                self.length_of_stay.tally(charging_time)
                self.power_mon.tally(self.power)
                # self.set_mode("PROC")
                self.hold(charging_time)
                self.car.activate()

    inter_arr_time_distr = sim.Exponential(number_of_EVSE / ev_arrival_rate)
    energy_request_distr = sim.Exponential(1 / energy_req_rate)

    # https://www.salabim.org/manual/Reference.html#environment
    app = sim.App(
        trace=False,  # defines whether to trace or not
        random_seed=random_seed,  # if “*”, a purely random value (based on the current time)
        time_unit=time_unit,  # defines the time unit used in the simulation
        name="Charging Station",  # name of the simulation
        do_reset=True,  # defines whether to reset the simulation when the run method is called
        yieldless=True,  # defines whether the simulation is yieldless or not
    )

    # Instantiate and activate the client generator
    EV_Generator(name="Electric Vehicles Generator")

    # Create Monitors

    # Create Queue and set monitor to stats_only
    # https://www.salabim.org/manual/Queue.html
    waitingline = sim.Queue(name="Waiting EV's", monitor=True)
    # waitingline.length_of_stay.monitor(value=True)
    waitingline.length.reset_monitors(stats_only=True)
    waitingline.length_of_stay.reset_monitors(stats_only=True)

    # Instantiate the EVSE's, list comprehension
    EVSE_facility = [EVSE() for _ in range(number_of_EVSE)]

    # Execute Simulation
    app.run(till=sim_time)

    total_evse_stay = sum(x.length_of_stay for x in EVSE_facility).mean()
    total_evse_lngt = sum(x.length for x in EVSE_facility).mean()

    # Return results
    return {
        "run": run,
        "lambda": ev_arrival_rate,
        "mu": energy_req_rate,
        "c": number_of_EVSE,
        "Wq": waitingline.length_of_stay.mean(),
        "Lq": waitingline.length.mean(),
        "Ws": total_evse_stay + waitingline.length_of_stay.mean(),
        "Ls": total_evse_lngt + waitingline.length.mean(),
    }


# function to run simulation X times
# returns a DataFrame with the results
def sim_x_facility(
    ev_arrival_rate,
    energy_req_rate,
    number_of_EVSE=1,
    sim_time=50000,
    number_of_simulations=30,
    verbose=False,
):
    sim_runs = []

    # Run simulation X times with different random seeds
    for i in range(number_of_simulations):
        sim_runs.append(
            sim_facility(
                ev_arrival_rate=ev_arrival_rate,
                energy_req_rate=energy_req_rate,
                number_of_EVSE=number_of_EVSE,
                sim_time=sim_time,
                random_seed=i,
                run=i,
            )
        )
        if verbose:
            print(f"EVSE's {number_of_EVSE}, run {i} completed at {datetime.now()}")

    # Concatenate all runs
    return pd.DataFrame(sim_runs)

## Run Simulation

In [37]:
# %%script false --no-raise-error # comment out this line to run the simulation CTRL + /
# Initialize an empty list
# Execute the simulation
dfs = []

for j in sim_evse:
    df = sim_x_facility(
    ev_arrival_rate=j * ev_arrival_rate,
    energy_req_rate=energy_req_rate,
    number_of_EVSE=j,
    sim_time=sim_time,
    number_of_simulations=sim_reps,
    verbose=False,
)
    # Append df to dfs
    dfs.append(df)

# Concatenate all DataFrames in dfs
df_total = pd.concat(dfs, ignore_index=True)

print(df_total)

df_total.to_csv("./sim_results/" + "R4" + sim_session_name)

EVSE's 1, run 0 completed at 2023-12-04 22:43:34.480449
EVSE's 2, run 0 completed at 2023-12-04 22:43:35.062438
EVSE's 3, run 0 completed at 2023-12-04 22:43:35.706478
EVSE's 4, run 0 completed at 2023-12-04 22:43:36.325829
EVSE's 5, run 0 completed at 2023-12-04 22:43:36.765602
EVSE's 6, run 0 completed at 2023-12-04 22:43:37.166910
EVSE's 7, run 0 completed at 2023-12-04 22:43:37.630499
EVSE's 8, run 0 completed at 2023-12-04 22:43:38.015997
EVSE's 9, run 0 completed at 2023-12-04 22:43:38.372499
   run    lambda        mu  c        Wq        Lq        Ws        Ls
0    0  0.666667  0.833333  1  4.041131  2.644247  5.260366  3.442033
1    0  1.333333  0.833333  2  0.224464  0.144106  1.422551  0.913089
2    0  2.000000  0.833333  3  0.022509  0.014661  1.234675  0.804185
3    0  2.666667  0.833333  4  0.004473  0.002914  1.217917  0.793270
4    0  3.333333  0.833333  5  0.001511  0.000984  1.214428  0.790998
5    0  4.000000  0.833333  6  0.000667  0.000435  1.214105  0.790787
6    0

In [1]:
# import matplotlib.pyplot as plt

# plt.plot(*waitingline.length.tx(), linewidth=1, color="red")
# plt.show()

# Print server statistics
I = monEVSE_IDLE.mean()
P = monEVSE_PROC.mean()
R = (P * monEVSE_PROC.number_of_entries()) / (
    (I * monEVSE_IDLE.number_of_entries()) + (P * monEVSE_PROC.number_of_entries())
)

print(f"IDLE: {I}, PROC: {P}, UTIL: {R}")

def prtEVSE(evse):
    print(
        f"name: {evse.proc.name()}, seq: {evse.proc.sequence_number()}, mean: {evse.proc.mean()}"
    )


[prtEVSE(x) in ChargingStations for x in ChargingStations]

print(f"Total: {sum(x.proc for x in ChargingStations).mean()}")
ChargingStations[0].status.print_histogram(values=True)
ChargingStations[0].mode.print_histogram(values=True)

IDLE: 0.062278801977539544
PROC: 1.193105570597679
UTIL: 0.9503906505943
name: TST.1, seq: 1, mean: 1.193105570597679
Total: 1.193105570597679
Histogram of chargingstation.1.status
duration         50000    

value                     duration     %
passive                   2480.565   5.0 ***
scheduled                47519.435  95.0 ****************************************************************************

Histogram of chargingstation.1.mode
duration         50000    

value                     duration     %
IDLE                      2480.565   5.0 ***
PROC                     47519.435  95.0 ****************************************************************************

